# Setting up Compass, Sass and Django

Brandon Rhodes gives a [great tutorial](http://rhodesmill.org/brandon/2011/adding-compass/) on how to add Compass to a [Django](https://www.djangoproject.com/) project. In that tutorial, the first step is to create a `compass` folder, and inside that folder, run an `install.sh` script to create the gems required for Compass.  I had trouble with the line inside this script that looks like this:

```shell
BASE=$(dirname $(readlink -f $(which "$0")))
```

For some reason on my Mac, `readlink` was missing. Instead, I [found](http://stackoverflow.com/questions/59895/can-a-bash-script-tell-what-directory-its-stored-in) a different way of obtaining the folder in which a script resides, and so my `install.sh` looks like this:

```shell
#!/bin/bash
# compass/install.sh - install Compass under the "./Gem" directory

if ! which gem >/dev/null ;then
    echo 'Error: no "gem" command available'
    echo 'Please install Ruby.'
    exit 1
fi
BASE=$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )
cd $BASE  # the directory where this script lives
gem install -i Gem compass
gem install -i Gem compass-susy-plugin
```

The same occurs in the second script, `compass.sh`:

```shell
#!/bin/bash
# compass/compass.sh - properly invoke the "Compass" program

BASE=$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )
export GEM_HOME=$BASE/Gem
export RUBYLIB=$BASE/Gem/lib
$BASE/Gem/bin/compass "$@"
```

The remainder of the tutorial remains the same.